# Utilisation du GPU sur un ordinateur de l'Enseirb

## 1) Vérifier qu'un GPU est présent et configuré

Dans un terminal, entrer la commande `nvidia-smi`  
Si vous obtenez quelque chose de similaire au résultat suivant, alors vous avez bien un GPU disponible en local sur votre ordinateur.

![Sortie nvidia-smi](nvidia-smi.png)

## 2) Configurer l'environnement Python/PyTorch

Afin de pouvoir optimiser les paramètres d'un réseau de neurones avec ce GPU, il faut disposer d'un environnement Python où PyTorch est installé. Un tel environnement est disponible à l'adresse : `/net/npers/gbourmaud/pytorch_env/bin/python`

Afin de pouvoir accéder à cet environnement, il est conseillé d'utiliser VSCode.  
Ouvrir VSCode depuis un terminal : `code &`

Pour que VSCode utilise le bon environnement python, il faut faire Ctrl+Shift+P et taper "Python: Select Interpreter" et finalement écrire l'adresse `/net/npers/gbourmaud/pytorch_env/bin/python`

Vous pouvez tester votre configuration avec le code suivant. Vous devriez constater que le calcul sur GPU est plus rapide que celui sur CPU.

In [ ]:
import torch
import time

def main():

    print("PyTorch version:", torch.__version__)
    gpu_available = torch.cuda.is_available()
    print("Is CUDA (GPU) available?", gpu_available)

    if gpu_available:
        print("GPU Name:", torch.cuda.get_device_name(0))
        print("CUDA Device Count:", torch.cuda.device_count())

    size = 3000  # Increase for more dramatic differences
    print(f"Matrix size: {size}x{size}")

    #GPU benchmark
    device = torch.device('cuda')

    a = torch.rand(size, size, device=device) #warmup
    b = torch.rand(size, size, device=device) #warmup
    _ = torch.mm(a, b) #warmup

    start = time.time()
    c= []
    for _ in range(10):
        a = torch.rand(size, size, device=device)
        b = torch.rand(size, size, device=device)
        c.append(torch.mm(a, b))

    t_GPU = time.time() - start
    print(f"GPU time: {t_GPU:.4f} seconds")

    #CPU benchmark
    device = torch.device('cpu')

    a = torch.rand(size, size, device=device) #warmup
    b = torch.rand(size, size, device=device) #warmup
    _ = torch.mm(a, b) #warmup

    start = time.time()
    c= []
    for _ in range(10):
        a = torch.rand(size, size, device=device)
        b = torch.rand(size, size, device=device)
        c.append(torch.mm(a, b))
    t_CPU = time.time() - start
    print(f"CPU time: {t_CPU:.4f} seconds")

    speedup = t_CPU / t_GPU
    print(f"Speedup (CPU / GPU): {speedup:.2f}x")

if __name__ == "__main__":
    main()